In [1]:
import os
os.chdir("/home/atulgang/Thesis")


In [2]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [3]:
import nltk
from nltk.tokenize import word_tokenize
from stop_words import get_stop_words
from nltk.stem import WordNetLemmatizer
from gensim import corpora, models
import gensim
import glob2
from itertools import chain
import os
import numpy as np
import itertools
import cPickle as pickle
from  scipy.stats import rankdata
import copy
from openpyxl import Workbook
from openpyxl.compat import range
from openpyxl.utils import get_column_letter
from openpyxl import load_workbook
import pandas as pd
from scipy.stats import hmean
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer

2018-05-06 15:40:26,806 : INFO : 'pattern' package not found; tag filters are not available for English


In [4]:

# WordNet only cares about 5 parts of speech.
# The other parts of speech will be tagged as nouns.

part = {
    'N' : 'n',
    'V' : 'v',
    'J' : 'a',
    'S' : 's',
    'R' : 'r'
}

wnl = WordNetLemmatizer()

def convert_tag(penn_tag):
    '''
    convert_tag() accepts the **first letter** of a Penn part-of-speech tag,
    then uses a dict lookup to convert it to the appropriate WordNet tag.
    '''
    if penn_tag in part.keys():
        return part[penn_tag]
    else:
        # other parts of speech will be tagged as nouns
        return 'n'


def tag_and_lem(element):
    '''
    tag_and_lem() accepts a token list, tags, converts tags,
    lemmatizes, and returns a string
    '''
    # list of tuples [('token', 'tag'), ('token2', 'tag2')...]
    sent = pos_tag(element) # must tag in context
    return [ wnl.lemmatize(sent[k][0], convert_tag(sent[k][1][0])) for k in range(len(sent))]

In [5]:
# ignores everything except english alphabet and  
def only_alphabet(text):
    return ''.join(i for i in text if (ord(i)<123 and ord(i)>96) or (ord(i)<91 and ord(i)>64) or ord(i)==32) 

full_filenames = []
x= ['LegalAdhocTask/ConsumerCourtAdhocdata/DCDRC/**/*.txt','LegalAdhocTask/ConsumerCourtAdhocdata/NCDRC/**/*.txt','LegalAdhocTask/ConsumerCourtAdhocdata/SCDRC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/DelhiHC/**/*.txt','LegalAdhocTask/SC-HCAdhocData/JharkhandHC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/JodhpurHC/**/*.txt' ,'LegalAdhocTask/SC-HCAdhocData/KolkataHC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/SupremeCourt/**/*.txt']
for glob_filenames in x:
    glob_filenames = os.path.normpath(glob_filenames)
    for filename in sorted(glob2.glob(glob_filenames)):
        full_filenames.append(filename)
            
filenames = []
x= ['LegalAdhocTask/ConsumerCourtAdhocdata/DCDRC/**/*.txt','LegalAdhocTask/ConsumerCourtAdhocdata/NCDRC/**/*.txt','LegalAdhocTask/ConsumerCourtAdhocdata/SCDRC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/DelhiHC/**/*.txt','LegalAdhocTask/SC-HCAdhocData/JharkhandHC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/JodhpurHC/**/*.txt' ,'LegalAdhocTask/SC-HCAdhocData/KolkataHC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/SupremeCourt/**/*.txt']
for glob_filenames in x:
    glob_filenames = os.path.normpath(glob_filenames)
    for filename in sorted(glob2.glob(glob_filenames)):
        filenames.append(os.path.basename(filename))

In [6]:
en_stop = set(get_stop_words('en'))

# "yield" for each file return token list  i.e list of lists
def files_to_tokens(glob_filenames):
    glob_filenames = os.path.normpath(glob_filenames)
    for filename in sorted(glob2.glob(glob_filenames)):
        f = open(filename)
        #print filename
        # read the whole file as lowercase string
        string = (f.read()).lower()
        try:
            temp = word_tokenize(string)
        except:
            string = string.decode("latin-1")
            temp = word_tokenize(string)
        
        tokens = []
        # tokenize that string
        for word in temp:
            w =  only_alphabet(word).lower()
            if w not in en_stop:
                if w:
                    tokens.append(w)
        tokens = tag_and_lem(tokens)
        yield tokens
        f.close()

        
# yields token list for files specific to courts; needed for creating dictionaries
class texts:
    def DCDRC(self):
        return files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/DCDRC/**/*.txt')
    def NCDRC(self):
        return files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/NCDRC/**/*.txt')
    def SCDRC(self):
        return files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/SCDRC/**/*.txt')
    def DelhiHC(self):
        return files_to_tokens('LegalAdhocTask/SC-HCAdhocData/DelhiHC/**/*.txt')
    def JharkhandHC(self):
        return files_to_tokens('LegalAdhocTask/SC-HCAdhocData/JharkhandHC/**/*.txt')
    def JodhpurHC(self):
        return files_to_tokens('LegalAdhocTask/SC-HCAdhocData/JodhpurHC/**/*.txt')
    def KolkataHC(self):
        return files_to_tokens('LegalAdhocTask/SC-HCAdhocData/KolkataHC/**/*.txt')
    def SupremeCourt(self):
        return files_to_tokens('LegalAdhocTask/SC-HCAdhocData/SupremeCourt/**/*.txt')
    def everything(self):
        return chain(self.DCDRC(), self.NCDRC(), self.SCDRC(), self.DelhiHC(),
                     self.JharkhandHC(), self.JodhpurHC(), self.KolkataHC(), self.SupremeCourt())

# yields bow for each file - tuples id,fq ; needed to train models   
class my_corpus:    
    def DCDRC(self):
        for text in files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/DCDRC/*.txt'):
            yield dictionary.doc2bow(text)
    def NCDRC(self):
        for text in files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/NCDRC/**/*.txt'):
            yield dictionary.doc2bow(text)            
    def SCDRC(self):
        for text in files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/SCDRC/**/*.txt'):
            yield dictionary.doc2bow(text)
    def DelhiHC(self):
        for text in files_to_tokens('LegalAdhocTask/SC-HCAdhocData/DelhiHC/**/*.txt'):
            yield dictionary.doc2bow(text)
    def JharkhandHC(self):
        for text in files_to_tokens('LegalAdhocTask/SC-HCAdhocData/JharkhandHC/**/*.txt'):
            yield dictionary.doc2bow(text)
    def JodhpurHC(self):
        for text in  files_to_tokens('LegalAdhocTask/SC-HCAdhocData/JodhpurHC/**/*.txt'):
            yield dictionary.doc2bow(text)
    def KolkataHC(self):
        for text in  files_to_tokens('LegalAdhocTask/SC-HCAdhocData/KolkataHC/**/*.txt'):
            yield dictionary.doc2bow(text)
    def SupremeCourt(self):
        for text in  files_to_tokens('LegalAdhocTask/SC-HCAdhocData/SupremeCourt/**/*.txt'):
            yield dictionary.doc2bow(text)
    def everything(self):
        return chain(self.DCDRC(), self.NCDRC(), self.SCDRC(), self.DelhiHC(),
                     self.JharkhandHC(), self.JodhpurHC(), self.KolkataHC(), self.SupremeCourt())
        
        

In [7]:
#################################### laod the dictinary
dictionary = corpora.Dictionary.load('dict/lem10')

2018-04-29 16:48:42,171 : INFO : loading Dictionary object from dict/lem10
2018-04-29 16:48:42,224 : INFO : loaded dict/lem10


In [8]:
######################################## load BOW
corpus = corpora.MmCorpus('corpus/bow_lem10.mm')

2018-04-29 16:48:42,300 : INFO : loaded corpus index from corpus/bow_lem10.mm.index
2018-04-29 16:48:42,301 : INFO : initializing cython corpus reader from corpus/bow_lem10.mm
2018-04-29 16:48:42,301 : INFO : accepted corpus with 351985 documents, 169716 features, 99651009 non-zero entries


In [9]:
# load p_C
with open('p_C/p_C_lem10.pkl', 'rb') as ip:
    p_C = pickle.load(ip)

In [10]:
# load doc_len
with open('doc_len/doc_len_lem10.pkl', 'rb') as ip:
    doc_len = pickle.load(ip)

In [11]:
# get the query truth in dictionary
query_truth = {}
for i in range(1,11):
    query_truth[str(i)]=[]
    
    
f = open('LegalAdhocTask/Consumer.qrels')
lines = [line.rstrip('\n').split("\t") for line in f]
for line in lines:
    del line[1]
    query_truth[line[0]].append(line[1:])


In [12]:

import math
from six import iteritems
from six.moves import xrange


# Dirichlet parameter
MU = 2000


class Dirichlet(object):

    def __init__(self, corpus):
        self.corpus = corpus
        self.corpus_size = dictionary.num_docs

    def get_score(self, query, index, query_len):
        index_doc = dict(self.corpus[index])
        score = 0
        for word,freq in query:
            if word not in index_doc:
                continue
            score += freq*(math.log( 1 + index_doc[word]/ (MU * p_C[word]) ))
        return score + query_len * math.log(MU/(doc_len[index] + MU))

    def get_scores(self, query):
        scores = []
        query_len = 0
        for word, freq in query:
            query_len += freq
        for index in xrange(self.corpus_size):
            score = self.get_score(query, index, query_len)
            scores.append(score)
        return scores



In [13]:
%%time
D = Dirichlet(corpus)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 13.1 µs


In [14]:
%%time
results = copy.deepcopy(query_truth)

sim_list = [0]
rank_list = [0]
queries = [0]
for i,query in enumerate(files_to_tokens('LegalAdhocTask/q*.txt')):
    queries.append(query)
    sims = D.get_scores(dictionary.doc2bow(query))
    sim_list.append(sims)
    # rank of every document wrt similarity
    ranks = rankdata(sims, method='ordinal')
    ranks= len(ranks)+1 - ranks 
    rank_list.append(ranks)
    
    # update the query truth tuples with similarity score and the ranks
    for x in results[str(i+1)]:
        x.append(sims[filenames.index(x[0]+'.txt')])
        x.append(ranks[filenames.index(x[0]+'.txt')])
    print i

    
        

0
1
2
3
4
5
6
7
8
9
CPU times: user 8min 28s, sys: 17.8 s, total: 8min 46s
Wall time: 8min 46s


In [15]:
results

{'1': [['ConsumerCourt_DCDRC_100385', '1', 18.385870754414814, 400],
  ['ConsumerCourt_DCDRC_106530', '0', 15.103776845288401, 685],
  ['ConsumerCourt_DCDRC_107608', '1', 0.8539781391294383, 11992],
  ['ConsumerCourt_DCDRC_114291', '1', 15.623611982934293, 634],
  ['ConsumerCourt_DCDRC_114382', '1', 15.363849857393276, 658],
  ['ConsumerCourt_DCDRC_118185', '1', 31.14425353545366, 45],
  ['ConsumerCourt_DCDRC_130318', '0', 28.573386817644977, 69],
  ['ConsumerCourt_DCDRC_130570', '1', 45.6419706701278, 1],
  ['ConsumerCourt_DCDRC_131146', '1', 20.799219541272834, 271],
  ['ConsumerCourt_DCDRC_131717', '1', 10.596997807034814, 1451],
  ['ConsumerCourt_DCDRC_131741', '1', 20.79715217514127, 272],
  ['ConsumerCourt_DCDRC_131818', '1', 23.919467351818646, 155],
  ['ConsumerCourt_DCDRC_131950', '0', 21.276798701248012, 245],
  ['ConsumerCourt_DCDRC_131972', '0', 43.707049849915364, 2],
  ['ConsumerCourt_DCDRC_132932', '0', 23.40845072986802, 166],
  ['ConsumerCourt_DCDRC_133592', '1', 31.74

In [16]:
model_name = 'LM_Dir_lem10'

with open('py_results/' + model_name + 'sim_list.pkl', 'wb') as output:
    pickle.dump(sim_list, output, pickle.HIGHEST_PROTOCOL)
with open('py_results/' + model_name + 'rank_list.pkl', 'wb') as output:
    pickle.dump(rank_list, output, pickle.HIGHEST_PROTOCOL)


In [17]:
# Store results in 1st sheet


wb = Workbook()
dest_filename = 'result_excel/' + model_name + '.xlsx'
ws1 = wb.active
ws1.title = "ground_truth"
ws1.append(['Query','Filename', 'Relevance', 'Score', 'Rank'])
for key,value in results.iteritems():
    for i in value:
        ws1.append([int(key)]+ i)

wb.save(filename = dest_filename)

#Query wise (F score)

import pandas as pd

f = pd.ExcelFile(dest_filename)

from scipy.stats import hmean



places = 4
wb = load_workbook(dest_filename)
ws1 = wb.create_sheet(title="evaluation_Qwise")

ws1.append(['Query','Docs considered', 'Precision/Recall', 'Model', 1, 5 ,10, 25, 50, 100, 500, 1000])

k = [1.0, 5.0 ,10.0, 25.0, 50.0, 100.0, 500.0, 1000.0]
for sheet in f.sheet_names:
    if(sheet == "evaluation"):
        continue
    for q in np.arange(1,11):

        x = f.parse(sheet)
        x = x[x['Query'] == q]
        total_1 = (float)(x[x['Relevance'] == 1].shape[0])
        total_0 = (float)(x[x['Relevance'] == 0].shape[0])
        total = (float)(x.shape[0])
        
        ####  repitition due to complication in considering both 1 & 0 relevance
        # precision 1
        row = [q, 1, 'P', model_name ]
        for i in k:
            row.append(x[np.logical_and(x['Relevance'] == 1 , x['Rank'] <= i)].shape[0]/i)
        ws1.append(row);
        p1 = row;
        # precision 0
        row = [q, 0, 'P', model_name ]
        for i in k:
            row.append(x[np.logical_and(x['Relevance'] == 0 , x['Rank'] <= i)].shape[0]/i)    
        ws1.append(row);
        p0 = row;
        #precision 10
        row = [q, 10, 'P', model_name]
        for i in k:
            row.append(x[x['Rank'] <= i].shape[0]/i)  
        ws1.append(row);
        p10 = row;  
        
        
        # recall 1
        row = [q, 1, 'R', model_name]
        for i in k:
            row.append(x[np.logical_and(x['Relevance'] == 1 , x['Rank'] <= i)].shape[0]/total_1)
            row[-1] = round(row[-1],places)
        ws1.append(row);
        r1 = row;    
        # recall 0
        row = [q, 0, 'R', model_name ]
        for i in k:
            row.append(x[np.logical_and(x['Relevance'] == 0 , x['Rank'] <= i)].shape[0]/total_0)
            row[-1] = round(row[-1],places)
        ws1.append(row);
        r0 = row;
        #recall 10
        row = [q, 10, 'R', model_name ]
        for i in k:
            row.append(x[x['Rank'] <= i].shape[0]/total)
            row[-1] = round(row[-1],places)
        ws1.append(row);
        r10 = row;      


        # F 1
        row = [q, 1, 'F', model_name ]
        for i in range(-len(k), 0, 1):
            if p1[i] == 0.0 :
                row.append(0.0)
            else:
                row.append(hmean([p1[i],r1[i]]))
            row[-1] = round(row[-1],places)
        ws1.append(row);

        # F 0
        row = [q, 0, 'F', model_name]
        for i in range(-len(k), 0, 1):
            if p0[i] == 0.0:
                row.append(0.0)
            else:
                row.append(hmean([p0[i],r0[i]]))
            row[-1] = round(row[-1],places)
        ws1.append(row);

        # F 10
        row = [q, 10, 'F', model_name]
        for i in range(-len(k), 0, 1):
            if p10[i] == 0.0:
                row.append(0.0)
            else:
                row.append(hmean([p10[i],r10[i]]))
            row[-1] = round(row[-1],places)
        ws1.append(row);
        
wb.save(filename = dest_filename)


#Query wise Average precision
# precision sum
def p_sum(z):
    z = z.copy()  
    z.sort_values(inplace=True)
    result = 0
    for i,val in enumerate(z):
        result += (i+1)/float(val) 
    return result


import pandas as pd

f = pd.ExcelFile(dest_filename)

from scipy.stats import hmean


places = 4
wb = load_workbook(dest_filename)
ws1 = wb.create_sheet(title="AP_Qwise")

ws1.append(['Query','Docs considered', 'Precision/Recall', 'Model', 1, 5 ,10, 25, 50, 100, 500, 1000])


for sheet in f.sheet_names:    
    if(sheet == "evaluation"or sheet == "evaluation_Qwise"):
        continue
    for q in np.arange(1,11):
        x = f.parse(sheet)
        x = x[x['Query'] == q]
        total = {}
        total[1] = (float)(x[x['Relevance'] == 1].shape[0])
        total[0] = (float)(x[x['Relevance'] == 0].shape[0])
        total[10] = (float)(x.shape[0])
        
        for rel in [1,0]:
            # precision 1
            row = [q, rel, 'AP', model_name ]
            for i in [1.0, 5.0, 10.0, 25.0, 50.0, 100.0, 500.0, 1000.0]:
                row.append(p_sum(x[np.logical_and(x['Relevance'] == rel, x['Rank'] <= i)]['Rank'])/total[rel])
            ws1.append(row);

        row = [q, 10, 'AP', model_name ]
        for i in [1.0, 5.0, 10.0, 25.0, 50.0, 100.0, 500.0, 1000.0]:
            row.append(p_sum(x[x['Rank'] <= i]['Rank'])/total[10])
        ws1.append(row);
wb.save(filename = dest_filename)

In [4]:
def append_df_to_excel(filename, df, sheet_name='Sheet1', startrow=None,
                       truncate_sheet=False, 
                       **to_excel_kwargs):
    """
    Append a DataFrame [df] to existing Excel file [filename]
    into [sheet_name] Sheet.
    If [filename] doesn't exist, then this function will create it.

    Parameters:
      filename : File path or existing ExcelWriter
                 (Example: '/path/to/file.xlsx')
      df : dataframe to save to workbook
      sheet_name : Name of sheet which will contain DataFrame.
                   (default: 'Sheet1')
      startrow : upper left cell row to dump data frame.
                 Per default (startrow=None) calculate the last row
                 in the existing DF and write to the next row...
      truncate_sheet : truncate (remove and recreate) [sheet_name]
                       before writing DataFrame to Excel file
      to_excel_kwargs : arguments which will be passed to `DataFrame.to_excel()`
                        [can be dictionary]

    Returns: None
    """
    from openpyxl import load_workbook

    # ignore [engine] parameter if it was passed
    if 'engine' in to_excel_kwargs:
        to_excel_kwargs.pop('engine')

    writer = pd.ExcelWriter(filename, engine='openpyxl')

    try:
        # try to open an existing workbook
        writer.book = load_workbook(filename)

        # get the last row in the existing Excel sheet
        # if it was not specified explicitly
        if startrow is None and sheet_name in writer.book.sheetnames:
            startrow = writer.book[sheet_name].max_row

        # truncate sheet
        if truncate_sheet and sheet_name in writer.book.sheetnames:
            # index of [sheet_name] sheet
            idx = writer.book.sheetnames.index(sheet_name)
            # remove [sheet_name]
            writer.book.remove(writer.book.worksheets[idx])
            # create an empty sheet [sheet_name] using old index
            writer.book.create_sheet(sheet_name, idx)

        # copy existing sheets
        writer.sheets = {ws.title:ws for ws in writer.book.worksheets}
    except FileNotFoundError:
        # file does not exist yet, we will create it
        pass

    if startrow is None:
        startrow = 0

    # write out the new sheet
    df.to_excel(writer, sheet_name, startrow=startrow, index= False, header = False, **to_excel_kwargs)

    # save the workbook
    writer.save()

model_name = 'LM_Dir_lem10'
dest_filename = 'result_excel/' + model_name + '.xlsx'
        
f = pd.ExcelFile(dest_filename)
sec_dest = 'result_excel/all_results.xlsx'
f2 = pd.ExcelFile(sec_dest)
append_df_to_excel(sec_dest, f.parse(1), sheet_name= f2.sheet_names[0])
append_df_to_excel(sec_dest, f.parse(2), sheet_name= f2.sheet_names[1])

In [ ]:
############################################################

In [ ]:
def Top_x(x, R):
    R = list(R)
    x_names = []
    for i in range(x):
        x_names.append(filenames[R.index(i + 1)].strip('.txt'))
    return x_names
    
    

In [ ]:
def score_stat(word, freq, index):
    index_doc = dict(bm25.corpus[index])
    word = dictionary.doc2bow([word])[0][0]
    idf = bm25.idf[word] if bm25.idf[word] >= 0 else EPSILON * average_idf
    score = freq*((idf * index_doc[word] * (PARAM_K1 + 1)
              / (index_doc[word] + PARAM_K1 * (1 - PARAM_B + PARAM_B * doc_len[index] / bm25.avgdl))))
    return (index_doc[word], score)

In [ ]:
wb = Workbook()
dest_filename = 'Temp.xlsx'
ws = wb.active
ws.title = "Blank"
#ws.append([(2.2,2.2)])
wb.save(filename = dest_filename)
def common_words(q, F):
    temp = []
    for word, freq in dictionary.doc2bow(queries[q]) :
        temp.append((dictionary[word], freq, round(bm25.idf[word], 2)))
    temp.sort(key=lambda x: (x[2],x[1]), reverse=True)
    q_words = [i[0]  for i in temp]
    q_freq = [i[1]  for i in temp]
    index = filenames.index(F + '.txt')
    index_doc = dict(bm25.corpus[index])
    row = []
    for word, freq in  zip(q_words, q_freq):
        w = dictionary.doc2bow([word])[0][0]
        if w in index_doc: 
            f, s = score_stat(word, freq, index)
            row.append([word + ', ' + str(round(bm25.idf[w], 2)), freq, f,  round( (s*100)/sim_list[q][index], 1)])  
    row.sort(key = lambda x: (x[3],x[1],x[2]), reverse = True)
    wb = load_workbook('Temp.xlsx')
    ws = wb.create_sheet(title=str(q) + F)
    ws.append(['word,idf', 'Query freq', 'freq', '% score'])
    for i in row:
        ws.append(i)
    wb.save(filename = 'Temp.xlsx')


In [ ]:
f = open("toberead.txt", 'r') 
count = 0
q = 1
for i in f:
    if i == '\n':
        continue
    count = count + 1
    common_words(q, i.strip())
    if count % 5 == 0:
        q = q + 1


In [ ]:
for i in files_to_tokens(full_filenames[filenames.index(doc + '.txt')]):
    for j in i:
        print j

In [ ]:
doc = 'ConsumerCourt_SCDRC_30176'
corpus[]

In [ ]:
common_words(1, 'ConsumerCourt_DCDRC_41588')

In [ ]:
df = pd.DataFrame(index= None, columns=['words','pair'])


In [ ]:
df

In [ ]:
print df

In [ ]:
df.append(['lol', ('we','wew')])

In [ ]:
def uncommon_words(q, filename)

In [ ]:
# Store results in 1st sheet
wb = Workbook()
dest_filename = 'Explore_BM25.xlsx'
ws = wb.active
ws.title = "Blank"
#ws.append([(2.2,2.2)])
wb.save(filename = dest_filename)

wb = load_workbook('Explore_BM25.xlsx')

for q in range(1, 11):
    ws = wb.create_sheet(title="q" + str(q))
    #x_names = Top_x(10, rank_list[q])
    top_row = ['Doc Name', 'Rel', 'Doc Length', ' Rank', 'Score']
    sec_row = [' ']*5
    temp = []
    for word, freq in dictionary.doc2bow(queries[q]) :
        temp.append((dictionary[word], freq, round(bm25.idf[word], 2)))
    temp.sort(key=lambda x: (x[2],x[1]), reverse=True)
    ws.append(top_row + [i[0]  for i in temp])
    ws.append(sec_row + [str(i[1]) + ', ' + str(i[2]) for i in temp])
    q_words = [i[0]  for i in temp]
    q_freq = [i[1]  for i in temp]
    
    #continue
    doc_list = [i[0] for i in query_truth[str(q)]] + Top_x(10, rank_list[q])
    doc_list = list((set(doc_list)))
    doc_list.sort(key = lambda doc : rank_list[q][filenames.index(doc + '.txt')])
    doc_row = []
    for doc in doc_list:
        temp  = [doc]
        rel_dic = dict(query_truth[str(q)])
        if doc in rel_dic:
            temp.append(rel_dic[doc])
        else:
            temp.append('Not')
        temp.append(doc_len[filenames.index(doc + '.txt')])
        temp.append(rank_list[q][filenames.index(doc + '.txt')])
        temp.append(round(sim_list[q][filenames.index(doc + '.txt')], 2))
        doc_row.append(temp)

    for row in doc_row:
        index = filenames.index(row[0] + '.txt')
        index_doc = dict(bm25.corpus[index])
        
        for word, freq in  zip(q_words, q_freq):
            if dictionary.doc2bow([word])[0][0] in index_doc: 
                f, s = score_stat(word, freq, index)
                row.append( str(f) + ', ' +  str(round( (s*100)/sim_list[q][index], 1) ))
            else:
                row.append('X')
        ws.append(row)
        ws.append([])        
wb.save(filename = dest_filename)

In [ ]:
# Store results in 1st sheet
wb = Workbook()
dest_filename = 'Explore_BM25_uncom.xlsx'
ws = wb.active
ws.title = "Blank"
#ws.append([(2.2,2.2)])
wb.save(filename = dest_filename)



wb = load_workbook('Explore_BM25_uncom.xlsx')

for q in range(1, 11):
    ws = wb.create_sheet(title="q" + str(q))
    #x_names = Top_x(10, rank_list[q])
    top_row = ['Doc Name', 'Rel', 'Doc Length', ' Rank', 'Score']
    sec_row = [' ']*5
    temp = []
    for word, freq in dictionary.doc2bow(queries[q]) :
        temp.append((dictionary[word], freq, round(bm25.idf[word], 2)))
    temp.sort(key=lambda x: (x[2],x[1]), reverse=True)
    ws.append(top_row + [i[0]  for i in temp])
    ws.append(sec_row + [str(i[1]) + ', ' + str(i[2]) for i in temp])
    q_words = [i[0]  for i in temp]
    q_freq = [i[1]  for i in temp]
    
    #continue
    doc_list = [i[0] for i in query_truth[str(q)]] + Top_x(10, rank_list[q])
    doc_list = list((set(doc_list)))
    doc_list.sort(key = lambda doc : rank_list[q][filenames.index(doc + '.txt')])
    doc_row = []
    for doc in doc_list:
        temp  = [doc]
        rel_dic = dict(query_truth[str(q)])
        if doc in rel_dic:
            temp.append(rel_dic[doc])
        else:
            temp.append('Not')
        temp.append(doc_len[filenames.index(doc + '.txt')])
        temp.append(rank_list[q][filenames.index(doc + '.txt')])
        temp.append(round(sim_list[q][filenames.index(doc + '.txt')], 2))
        doc_row.append(temp)

    for row in doc_row:
        index = filenames.index(row[0] + '.txt')
        index_doc = dict(bm25.corpus[index])
        
#         for word, freq in  zip(q_words, q_freq):
#             if dictionary.doc2bow([word])[0][0] in index_doc: 
#                 f, s = score_stat(word, freq, index)
#                 row.append( str(f) + ', ' +  str(round( (s*100)/sim_list[q][index], 1) ))
#             else:
#                 row.append('X')
        #ws.append(row)
        
        sec_row = [' ']*5
        temp = []
        for word, freq in index_doc.iteritems():
            if dictionary[word] not in q_words:
                temp.append((dictionary[word], freq, round(bm25.idf[word], 2)))
        ws.append(row + [i[0]  for i in temp])
        ws.append(sec_row + [str(i[1]) + ', ' + str(i[2]) for i in temp])
        ws.append([])        
wb.save(filename = dest_filename)

In [ ]:
# Store results in 1st sheet
wb = Workbook()
dest_filename = 'Explore_BM25_all.xlsx'
ws = wb.active
ws.title = "Blank"
#ws.append([(2.2,2.2)])
wb.save(filename = dest_filename)



wb = load_workbook('Explore_BM25_all.xlsx')

for q in range(1, 11):
    ws = wb.create_sheet(title="q" + str(q))
    #x_names = Top_x(10, rank_list[q])
    top_row = ['Doc Name', 'Rel', 'Doc Length', ' Rank', 'Score']
    sec_row = [' ']*5
    temp = []
    for word, freq in dictionary.doc2bow(queries[q]) :
        temp.append((dictionary[word], freq, round(bm25.idf[word], 2)))
    temp.sort(key=lambda x: (x[2],x[1]), reverse=True)
    ws.append(top_row + [i[0]  for i in temp])
    ws.append(sec_row + [str(i[1]) + ', ' + str(i[2]) for i in temp])
    q_words = [i[0]  for i in temp]
    q_freq = [i[1]  for i in temp]
    
    #continue
    doc_list = [i[0] for i in query_truth[str(q)]] + Top_x(10, rank_list[q])
    doc_list = list((set(doc_list)))
    doc_list.sort(key = lambda doc : rank_list[q][filenames.index(doc + '.txt')])
    doc_row = []
    for doc in doc_list:
        temp  = [doc]
        rel_dic = dict(query_truth[str(q)])
        if doc in rel_dic:
            temp.append(rel_dic[doc])
        else:
            temp.append('Not')
        temp.append(doc_len[filenames.index(doc + '.txt')])
        temp.append(rank_list[q][filenames.index(doc + '.txt')])
        temp.append(round(sim_list[q][filenames.index(doc + '.txt')], 2))
        doc_row.append(temp)

    for row in doc_row:
        index = filenames.index(row[0] + '.txt')
        index_doc = dict(bm25.corpus[index])
        
        for word, freq in  zip(q_words, q_freq):
            if dictionary.doc2bow([word])[0][0] in index_doc: 
                f, s = score_stat(word, freq, index)
                row.append( str(f) + ', ' +  str(round( (s*100)/sim_list[q][index], 1) ))
            else:
                row.append('X')
        ws.append(row)
        
        sec_row = [' ']*5
        temp = []
        for word, freq in index_doc.iteritems():
            if dictionary[word] not in q_words:
                temp.append((dictionary[word], freq, round(bm25.idf[word], 2)))
        ws.append(sec_row + [i[0]  for i in temp])
        ws.append(sec_row + [str(i[1]) + ', ' + str(i[2]) for i in temp])
        ws.append([])        
wb.save(filename = dest_filename)